In [ ]:
# data_generator.py
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import warnings
import json
import os

warnings.filterwarnings('ignore')

# 设置随机种子确保可重复性
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

def load_data(csv_path):
    """加载硬度数据集"""
    df = pd.read_csv(csv_path, index_col=0)
    
    # 提取硬度标签
    HV_values = df.loc[:, "HV"].values
    
    # 提取合金成分数据 (21种元素)
    X_compositions = df.iloc[:, 18:39]
    
    # 过滤出至少有一个非零值的元素列
    non_zero_cols = (X_compositions != 0).any(axis=0)
    X_compositions = X_compositions.loc[:, non_zero_cols]
    element_names = X_compositions.columns.tolist()
    
    # 按原子序数排序
    ATOMIC_NUMBER = {
        'Li': 3, 'Mg': 12, 'Al': 13, 'Si': 14, 'Sc': 21,
        'Ti': 22, 'V': 23, 'Cr': 24, 'Mn': 25, 'Fe': 26,
        'Co': 27, 'Ni': 28, 'Cu': 29, 'Zn': 30, 'Zr': 40,
        'Nb': 41, 'Mo': 42, 'Sn': 50, 'Hf': 72, 'Ta': 73,
        'W': 74
    }
    
    # 只保留出现在数据中的元素
    present_elements = [el for el in element_names if el in ATOMIC_NUMBER]
    # 按原子序数排序
    sorted_elements = sorted(present_elements, key=lambda x: ATOMIC_NUMBER.get(x, float('inf')))
    
    # 重新排列DataFrame列顺序
    X_compositions = X_compositions[sorted_elements]
    
    print(f"数据集大小: {len(df)}")
    print(f"元素列表（按原子序数排序）: {sorted_elements}")
    print(f"HV值范围: [{HV_values.min():.2f}, {HV_values.max():.2f}]")
    
    return X_compositions, HV_values, sorted_elements

# 2. 元素嵌入特征提取 - 改为类以复用模型
class ElementEmbedder:
    def __init__(self, model_path, device=None):
        if device is None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = device
            
        print(f"加载嵌入模型，使用设备: {self.device}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModel.from_pretrained(model_path).to(self.device)
        self.model.eval()
        
    def get_element_embedding(self, element_name):
        """获取单个元素的嵌入向量"""
        # Tokenize
        inputs = self.tokenizer(
            element_name,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=256
        ).to(self.device)
        
        # 获取嵌入
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        # 使用[CLS] token的嵌入
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        
        return cls_embedding.cpu().numpy().flatten()

# 3. 生成合金成分的加权嵌入矩阵
def generate_alloy_embedding_matrix(compositions, element_names, elements_embedding, max_elements):
    """
    为每个合金生成二维嵌入矩阵
    
    参数:
    - compositions: 合金成分DataFrame
    - element_names: 元素名称列表
    - elements_embedding: 元素嵌入字典
    - max_elements: 最大元素数量（用于统一矩阵大小）
    
    返回:
    - embedding_matrices: 形状为 (n_samples, max_elements, embedding_dim) 的numpy数组
    """
    # 检查嵌入维度
    test_embedding = elements_embedding[element_names[0]]
    embedding_dim = len(test_embedding)
    print(f"嵌入维度: {embedding_dim}")
    
    # 生成合金嵌入矩阵
    print("正在生成合金嵌入矩阵...")
    
    n_samples = len(compositions)
    # 预分配内存以提高效率
    embedding_matrices = np.zeros((n_samples, max_elements, embedding_dim))
    
    for idx in tqdm(range(n_samples), desc="处理合金"):
        composition = compositions.iloc[idx]
        
        for i in range(max_elements):
            element_name = element_names[i]
            weight = composition[element_name]
            
            # 如果含量小于0.1%，设为0（可根据需要调整阈值）
            if weight < 0.001:
                weight = 0
            
            # 加权嵌入向量
            element_embedding = elements_embedding[element_name]
            weighted_embedding = element_embedding * weight
            embedding_matrices[idx, i, :] = weighted_embedding
    
    print(f"嵌入矩阵形状: {embedding_matrices.shape}")
    return embedding_matrices

# 4. 示例展示函数
def show_example(compositions, element_names, elements_embedding, sample_idx=0, threshold=0.0001):
    """展示一个样本的示例"""
    print("\n" + "="*60)
    print("示例展示：单个合金的嵌入矩阵生成过程")
    print("="*60)
    
    composition = compositions.iloc[sample_idx]
    
    print(f"\n样本 {sample_idx} 的合金成分：")
    print("-"*40)
    
    # 显示非零元素及其含量
    nonzero_elements = []
    nonzero_weights = []
    total_composition = 0
    
    for i, element in enumerate(element_names):
        weight = composition.iloc[i]
        total_composition += weight
        if weight > threshold:
            nonzero_elements.append(element)
            nonzero_weights.append(weight)
            print(f"{element}: {weight:.6f} ({weight*100:.2f}%)")
    
    print(f"\n非零元素数量: {len(nonzero_elements)}")
    print(f"总成分含量: {total_composition:.6f}")
    
    # 显示第一个元素的嵌入向量（前10维）
    if nonzero_elements:
        first_element = nonzero_elements[0]
        embedding = elements_embedding[first_element]
        print(f"\n元素 '{first_element}' 的嵌入向量（前10维）:")
        print(f"[{', '.join([f'{x:.7f}' for x in embedding[:10]])}]")
        print(f"... 总共 {len(embedding)} 维")
        
        # 计算加权嵌入
        weight = nonzero_weights[0]
        weighted_embedding = embedding[:10] * weight
        
        print(f"\n加权后嵌入向量（前10维）:")
        print(f"[{', '.join([f'{x:.7f}' for x in weighted_embedding])}]")
        print(f"使用权重: {weight:.6f}")
        
        # 详细计算验证
        print(f"\n详细计算验证:")
        for i in range(3):  # 只显示前3个值的详细计算
            original = embedding[i]
            weighted = original * weight
            print(f"  dim{i}: {original:.7f} × {weight:.6f} = {weighted:.7f}")
    
    return nonzero_elements, nonzero_weights

# 5. 主函数
def main():
    # 设置路径
    csv_path = '..//data//HEAs_hardness_Data.csv'
    model_path = "..//DeBERTa//model_results_save//final_model"
    
    print("="*60)
    print("第一部分：数据生成和保存")
    print("="*60)
    
    # 1. 加载数据
    print("\n步骤1: 加载数据...")
    X_compositions, HV_values, element_names = load_data(csv_path)
    
    # 2. 初始化嵌入器
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"使用设备: {device}")
    embedder = ElementEmbedder(model_path, device)
    
    # 3. 获取所有元素嵌入
    print("\n步骤2: 生成元素嵌入向量...")
    elements_embedding = {}
    
    for element in tqdm(element_names, desc="处理元素"):
        embedding = embedder.get_element_embedding(element)
        elements_embedding[element] = embedding
    
    # 4. 展示示例
    show_example(X_compositions, element_names, elements_embedding)

    # 5. 生成完整的嵌入矩阵
    print("\n步骤3: 生成合金嵌入矩阵...")
    max_elements = len(element_names)  # 使用实际元素数量
    
    X_embedding = generate_alloy_embedding_matrix(
        X_compositions, 
        element_names,
        elements_embedding,
        max_elements=max_elements
    )
    
    # 6. 保存数据
    print("\n步骤4: 保存数据...")
    
    # 确保输出目录存在
    output_dir = './二维高熵合金数据'
    os.makedirs(output_dir, exist_ok=True)
    
    # 保存嵌入矩阵
    embedding_path = os.path.join(output_dir, 'alloy_embedding_matrices.npy')
    np.save(embedding_path, X_embedding)
    print(f"✓ 嵌入矩阵已保存到 {embedding_path}")
    
    # 保存HV值
    hv_path = os.path.join(output_dir, 'HV_values.npy')
    np.save(hv_path, HV_values)
    print(f"✓ HV值已保存到 {hv_path}")
    
    # 保存元素名称
    element_path = os.path.join(output_dir, 'element_names.txt')
    with open(element_path, 'w', encoding='utf-8') as f:
        for name in element_names:
            f.write(name + '\n')
    print(f"✓ 元素名称已保存到 {element_path}")
    
    # 保存数据信息
    data_info = {
        'n_samples': len(X_embedding),
        'max_elements': max_elements,
        'embedding_dim': X_embedding.shape[2],
        'HV_range': [float(HV_values.min()), float(HV_values.max())],  # 确保可序列化
        'element_count': len(element_names),
        'element_names': element_names
    }
    
    info_path = os.path.join(output_dir, 'data_info.json')
    with open(info_path, 'w', encoding='utf-8') as f:
        json.dump(data_info, f, indent=2, ensure_ascii=False)
    print(f"✓ 数据信息已保存到 {info_path}")
    
    # 7. 显示数据统计信息
    print("\n步骤5: 数据统计信息")
    print("-"*40)
    print(f"样本数量: {len(X_embedding)}")
    print(f"嵌入矩阵形状: {X_embedding.shape}")
    print(f"每个合金的元素数: {max_elements}")
    print(f"每个元素的嵌入维度: {X_embedding.shape[2]}")
    print(f"HV值统计:")
    print(f"  最小值: {HV_values.min():.2f}")
    print(f"  最大值: {HV_values.max():.2f}")
    print(f"  平均值: {HV_values.mean():.2f}")
    print(f"  标准差: {HV_values.std():.2f}")
    
    # 8. 验证保存的数据
    print("\n步骤6: 验证保存的数据...")
    try:
        loaded_embedding = np.load(embedding_path)
        loaded_hv = np.load(hv_path)
        print(f"✓ 嵌入矩阵验证: 加载形状 {loaded_embedding.shape}，与原始形状一致")
        print(f"✓ HV值验证: 加载 {len(loaded_hv)} 个值，与原始数量一致")
    except Exception as e:
        print(f"✗ 数据验证失败: {e}")
    
    print("\n" + "="*60)
    print("数据生成完成！可以使用第二部分代码进行CNN训练。")
    print("="*60)

# 运行主函数
if __name__ == "__main__":
    main()

第一部分：数据生成和保存

步骤1: 加载数据...
数据集大小: 483
元素列表（按原子序数排序）: ['Al', 'Si', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Zr', 'Nb', 'Mo', 'Sn', 'Hf', 'Ta', 'W']
HV值范围: [109.00, 1084.00]
使用设备: cuda
加载嵌入模型，使用设备: cuda

步骤2: 生成元素嵌入向量...


处理元素: 100%|██████████| 19/19 [00:00<00:00, 22.17it/s]



示例展示：单个合金的嵌入矩阵生成过程

样本 0 的合金成分：
----------------------------------------
Al: 0.016983 (1.70%)
Mn: 0.327672 (32.77%)
Ni: 0.327672 (32.77%)
Cu: 0.327672 (32.77%)

非零元素数量: 4
总成分含量: 1.000000

元素 'Al' 的嵌入向量（前10维）:
[1.0260553, 0.4972566, -0.3054150, -1.3371440, 2.2887373, -0.8514645, -2.0911560, 0.4040970, -0.1876834, -1.0278958]
... 总共 768 维

加权后嵌入向量（前10维）:
[0.0174255, 0.0084449, -0.0051869, -0.0227087, 0.0388697, -0.0144604, -0.0355141, 0.0068628, -0.0031874, -0.0174568]
使用权重: 0.016983

详细计算验证:
  dim0: 1.0260553 × 0.016983 = 0.0174255
  dim1: 0.4972566 × 0.016983 = 0.0084449
  dim2: -0.3054150 × 0.016983 = -0.0051869

步骤3: 生成合金嵌入矩阵...
嵌入维度: 768
正在生成合金嵌入矩阵...


处理合金: 100%|██████████| 483/483 [00:00<00:00, 2945.11it/s]


嵌入矩阵形状: (483, 19, 768)

步骤4: 保存数据...
✓ 嵌入矩阵已保存到 ./二维高熵合金数据\alloy_embedding_matrices.npy
✓ HV值已保存到 ./二维高熵合金数据\HV_values.npy
✓ 元素名称已保存到 ./二维高熵合金数据\element_names.txt
✓ 数据信息已保存到 ./二维高熵合金数据\data_info.json

步骤5: 数据统计信息
----------------------------------------
样本数量: 483
嵌入矩阵形状: (483, 19, 768)
每个合金的元素数: 19
每个元素的嵌入维度: 768
HV值统计:
  最小值: 109.00
  最大值: 1084.00
  平均值: 477.80
  标准差: 211.10

步骤6: 验证保存的数据...
✓ 嵌入矩阵验证: 加载形状 (483, 19, 768)，与原始形状一致
✓ HV值验证: 加载 483 个值，与原始数量一致

数据生成完成！可以使用第二部分代码进行CNN训练。
